# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Study data files
part_one = "Resources/citiesall.csv"
part_one = pd.read_csv(part_one)
part_one

FileNotFoundError: [Errno 2] File b'Resources/citiesall.csv' does not exist: b'Resources/citiesall.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = part_one[["Latitude", "Longitude"]]
humi = part_one["Humidity(%)"].astype(float)


In [6]:
part_one = part_one.dropna()


In [8]:
fig = gmaps.figure(center=(20.0,0.0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humi, dissipating = False, max_intensity=50, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#ideal vacation conditions:1. max temperature 75<x<90 2. Cloudiness <20% 3. Humidity <40%


In [9]:
ideal_df=part_one[(part_one['Temperature(F)']>75) & (part_one['Temperature(F)']<90) & (part_one['Humidity(%)']<40) & (part_one['Cloudiness(%)']<20)]
ideal_df.reset_index()

,index,City,Country,Temperature(F),Latitude,Longitude,Humidity(%),Cloudiness(%),Windspeed(mph)
0,40,ponta do sol,AU,87.22,-33.65,115.33,23.0,0.0,10.83
1,355,esperanza,AR,78.98,-33.30,-66.34,36.0,0.0,16.06
2,407,saleaula,AR,77.14,-42.77,-65.04,17.0,10.0,15.21
3,448,puerto madryn,AU,88.72,-33.37,116.15,23.0,0.0,6.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df=pd.DataFrame(ideal_df,columns=['City','Country','Latitude','Longitude'])
hotel_df['Hotel Name']= ""
hotel_df.reset_index()


,index,City,Country,Latitude,Longitude,Hotel Name
0,40,ponta do sol,AU,-33.65,115.33,
1,355,esperanza,AR,-33.30,-66.34,
2,407,saleaula,AR,-42.77,-65.04,
3,448,puerto madryn,AU,-33.37,116.15,


In [11]:
hotel_name=[]
target_type = "lodging"
radius = 5000

params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}
   # Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json=response.json()
    #pprint(response_json)
    hotel_name.append(response_json['results'][0]['name'])
    
    hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
    print(f"The closest hotel to {row[0]} is {response_json['results'][0]['name']}")    

The closest hotel to ponta do sol is Observatory Guest House
The closest hotel to esperanza is Huarpes Hotel
The closest hotel to saleaula is Hotel Tolosa
The closest hotel to puerto madryn is Collie Ridge


In [12]:
hotel_df
hotel_df_new=hotel_df.dropna()
hotel_df_new.reset_index()

,index,City,Country,Latitude,Longitude,Hotel Name
0,40,ponta do sol,AU,-33.65,115.33,Observatory Guest House
1,355,esperanza,AR,-33.30,-66.34,Huarpes Hotel
2,407,saleaula,AR,-42.77,-65.04,Hotel Tolosa
3,448,puerto madryn,AU,-33.37,116.15,Collie Ridge


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_new.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


fig

        


Figure(layout=FigureLayout(height='420px'))

In [17]:
fig = gmaps.figure(center=(20.0,0.0),zoom_level=1)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))